In [6]:
import sys
import os
# add parent directory (where "natsume" lives) to sys.path so I can import natsume
sys.path.append(r"C:\Users\WBS\Desktop\EXOPLANET WORK\006 NATSUME")
import natsume

import ttv_curvefit.curvefit as ttvfit  # custom-written
import ttvfast
from ttvfast import models

from tqdm import tqdm
import pandas as pd
import numpy as np
from astropy import units as u
from astropy.constants import M_earth
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import seaborn as sns

# Import CSV for TTVFast simulations
df = pd.read_csv(r"C:\Users\WBS\Desktop\EXOPLANET WORK\006 NATSUME\validation\1st_order_TTVFast\lithwick_TTVFast_params.csv")
df.head()

,e1,w1,e2,w2,Delta,j,P1,P2,m1,m2,mu1,mu2,validity1,validity2,valid
0,0.001283,68.375339,0.007207,138.362910,-0.043996,4,7,8.922707,2.772110,265.582724,0.000008,0.000798,181135.859840,336.694895,True
1,0.063464,109.286242,0.027547,332.358026,-0.032854,3,7,10.155034,706.150412,225.998388,0.002121,0.000679,8.019105,57.724774,False
2,0.053950,228.461969,0.004795,113.882986,-0.021484,3,7,10.274423,2819.784291,0.683979,0.008469,0.000002,1.010130,46857.789462,False
3,0.012312,191.112728,0.006331,108.626523,-0.034800,5,7,8.445502,58.226159,17.970338,0.000175,0.000054,562.441521,3544.163170,True
4,0.014892,179.591629,0.008487,94.519629,0.017762,5,7,8.905420,1.012663,142.226387,0.000003,0.000427,6965.506486,87.025596,False


In [7]:
# Copy dataframe for fitting results
df_fit = df.copy()
df_fit_cols = ["A1", "B1", "V1", "Pttv1", "phase1", "A2", "B2", "V2", "Pttv2", "phase2", "R2_1", "R2_2"]
df_fit[df_fit_cols] = np.nan
df_fit.head()

,e1,w1,e2,w2,Delta,j,P1,P2,m1,m2,...,V1,Pttv1,phase1,A2,B2,V2,Pttv2,phase2,R2_1,R2_2
0,0.001283,68.375339,0.007207,138.362910,-0.043996,4,7,8.922707,2.772110,265.582724,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.063464,109.286242,0.027547,332.358026,-0.032854,3,7,10.155034,706.150412,225.998388,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.053950,228.461969,0.004795,113.882986,-0.021484,3,7,10.274423,2819.784291,0.683979,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.012312,191.112728,0.006331,108.626523,-0.034800,5,7,8.445502,58.226159,17.970338,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.014892,179.591629,0.008487,94.519629,0.017762,5,7,8.905420,1.012663,142.226387,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# For storing TTVs
df_ttv_cols = ["t0", "ttv0", "t1", "ttv1"]
df_ttv = pd.DataFrame(index=range(len(df)), columns=df_ttv_cols, dtype=object)
df_ttv.head()

,t0,ttv0,t1,ttv1
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN


In [10]:
# Create TTV simulations and extract Amplitude and Period
gravity = 0.000295994511 # AU^3/day^2/M_sun
stellar_mass = 1.0       # M_sun

for i, row in tqdm(df.iterrows(), total=len(df)):
    # If invalid scenario, skip
    if row['valid'] == False:
        continue
    
    # Initialize variables
    P1 = row['P1']; P2 = row['P2']
    j = row['j']
    PTTV = 1/np.abs(j/P2 - (j-1)/P1)
    
    e1 = row['e1']; w1 = row['w1']
    e2 = row['e2']; w2 = row['w2']
    
    mu1 = row['mu1']; mu2 = row['mu2']
    
    # Create TTVFast planets
    planet1 = models.Planet(
        mass=mu1,            # M_sun
        period=P1,           # days
        eccentricity=e1,
        inclination=90,      # degrees
        longnode=0,          # degrees
        argument=90+w1,      # degrees
        mean_anomaly=0       # degrees
    )
    planet2 = models.Planet(
        mass=mu2,
        period=P2,
        eccentricity=e2,
        inclination=90,
        longnode=0,
        argument=90+w2,
        mean_anomaly=0,
    )
    planets = [planet1, planet2]
    
    begin_time = -P2 * 200
    Time = begin_time   # days
    dt = P1 / 100       # days
    Total = max(250, PTTV * 2.5)  # days
    
    results = ttvfast.ttvfast(planets, stellar_mass, Time, dt, Total,
                              input_flag=1)

    # Create TTV signal from TTVFast Output
    out = pd.DataFrame(results['positions']).transpose()
    out.columns = ['planet', 'epoch', 'times', 'rsky', 'vsky']
    
    table0 = out[(out.planet == 0) & (out.times > 0)]
    n0 = table0.epoch
    tn0 = table0.times

    table1 = out[(out.planet == 1) & (out.times > 0)]
    n1 = table1.epoch
    tn1 = table1.times

    # If no (more like less than 10) TTVFast output, skip
    len_threshold = 10
    if (len(n0) <= len_threshold) or (len(n1) <= len_threshold):
        continue
        
    time0, ttv0, ttv0_err = ttvfit.return_ttv(epochs=n0, midtransits=tn0)
    time1, ttv1, ttv1_err = ttvfit.return_ttv(epochs=n1, midtransits=tn1)
    
    # Make sure ttv0 and ttv1 actually are centered around O-C = 0
    ttv0 = ttv0 - (np.max(ttv0) + np.min(ttv0)) / 2
    ttv1 = ttv1 - (np.max(ttv1) + np.min(ttv1)) / 2
    
    # Get sinusoidal TTV model parameters
    popt_ttv0, perr_ttv0, r2_0 = ttvfit.characterize_ttv(time=time0, ttv=ttv0, ttv_err=ttv0_err,
                                                         expected_period=PTTV, method='lmfit')
    popt_ttv1, perr_ttv1, r2_1 = ttvfit.characterize_ttv(time=time1, ttv=ttv1, ttv_err=ttv1_err,
                                                         expected_period=PTTV, method='lmfit')

    # Write results to df_fit and df_ttv
    df_ttv.loc[i] = {
        "t0": time0,
        "ttv0": ttv0,
        "t1": time1,
        "ttv1": ttv1
    }
    df_fit.loc[i, df_fit_cols]  = [*popt_ttv0, *popt_ttv1, r2_0, r2_1]
    
    # # Plot TTV signal
    # print(f'Expected PTTV | validity1 | validity2 | R2 1,2')
    # print(f'{PTTV} d | {row["validity1"]} | {row["validity2"]} | {r2_0, r2_1}')
    
    # scaling = 1e+3
    # plt.figure(figsize=(10,4))
    # plt.scatter(time0, (ttv0 - popt_ttv0[0] - popt_ttv0[1]*time0) * scaling,
    #             s=25, c='black', label='Inner planet', zorder=1)
    # plt.scatter(time1, (ttv1 - popt_ttv1[0] - popt_ttv1[1]*time1) * scaling,
    #             s=25, c='blue', label='Outer planet', zorder=1)
    # plt.plot(time0, ttvfit.ttv_model(time0, 0, 0, *popt_ttv0[2:]) * scaling,
    #          c='gray', zorder=0)
    # plt.plot(time1, ttvfit.ttv_model(time1, 0, 0, *popt_ttv1[2:]) * scaling,
    #          c='cornflowerblue', zorder=0)
    
    # plt.title('TTV simulation (points) and sinusoidal fits (curves)')
    # plt.xlabel('Time [day]'); plt.ylabel('TTV [10$^{-3}$ day]')
    # plt.xlim(time0.min(), time0.max())
    
    # ax = plt.gca()
    # ax.yaxis.grid(True, linestyle=':', color='gray', alpha=0.5, zorder=-1) # Y-axis lines!
    # plt.legend(loc='upper right')

    # plt.xlim(0, Total)
    
    # plt.show()

100%|████████████████████████████████████████████████████████████████████████████| 15000/15000 [18:15<00:00, 13.69it/s]


In [11]:
# Export dataframes
df_ttv.to_csv("lithwick_TTVFast_outputs.csv", index=False)
df_fit.to_csv("lithwick_TTVFast_fits.csv", index=False)